In [ ]:
def plot_hitting_metrics_combined(df, athlete_id, metrics, facility_averages, pdf):
    """
    Plot historical progress of hitting metrics for an athlete on one page.

    Parameters:
    - df: Pandas DataFrame containing test data.
    - athlete_id: Athlete's ID (string or integer).
    - metrics: List of metrics to plot (e.g., ["Max Bat Speed", "Average EV"]).
    - facility_averages: Dictionary of facility average values grouped by level.
    - pdf: PdfPages object for saving plots.
    """
    # Filter athlete-specific data
    athlete_data = df[df['ID'] == athlete_id]

    # Check for "Hitting" in Workout Type
    hitting_data = athlete_data[athlete_data['Workout Type'] == 'Hitting']
    if hitting_data.empty:
        print(f"Skipping Hitting Metrics Plot for Athlete ID {athlete_id}: No hitting data available.")
        return  # Skip if no hitting data is available

    # Convert date column
    athlete_data["Date"] = pd.to_datetime(athlete_data["Date"], errors="coerce")
    athlete_data = athlete_data.dropna(subset=["Date"]).sort_values(by="Date")

    # Test sub-types and their colors
    test_sub_types = athlete_data["Test Sub-Type"].dropna().unique()
    sub_type_colors = {sub: plt.cm.tab10(i) for i, sub in enumerate(test_sub_types)}

    # Create a single figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    axes = axes.flatten()

    for i, metric in enumerate(metrics):
        ax = axes[i]

        # Plot athlete data by test sub-type
        for sub_type, color in sub_type_colors.items():
            sub_data = athlete_data[athlete_data["Test Sub-Type"] == sub_type]
            if not sub_data.empty and metric in sub_data.columns:
                valid_data = sub_data.dropna(subset=[metric])
                if not valid_data.empty:
                    ax.plot(
                        valid_data["Date"],
                        valid_data[metric],
                        marker="o",
                        linestyle="-",
                        label=sub_type,  # Ensure the legend tracks the sub-type
                        color=color,
                    )
                    # Annotate each data point
                    for _, row in valid_data.iterrows():
                        ax.text(
                            row["Date"],
                            row[metric],
                            f"{row[metric]:.1f}",
                            fontsize=8,
                            ha="center",
                            va="bottom",
                        )

        # Plot facility average
        athlete_level = athlete_data["Level"].iloc[-1]
        if metric in facility_averages and athlete_level in facility_averages[metric]:
            ax.axhline(
                facility_averages[metric][athlete_level],
                color="gray",
                linestyle="--",
                label="Facility Average",
            )

        # Customize subplot
        ax.set_title(f"{metric} Progress", fontsize=14, fontweight="bold")
        ax.set_xlabel("Date", fontsize=10)
        ax.set_ylabel(metric, fontsize=10)
        ax.grid(True, linestyle="--", alpha=0.7)

        # Format x-axis to show only test dates
        unique_dates = athlete_data.dropna(subset=[metric])["Date"].drop_duplicates()
        formatted_dates = unique_dates.dt.strftime("%m/%d/%Y")
        ax.set_xticks(unique_dates)
        ax.set_xticklabels(formatted_dates, rotation=45, ha="right", fontsize=8)

    # Add legend in the top-right corner (outside the plot area)
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(
        handles,
        labels,
        title="Test Sub-Type",
        loc="upper right",
        fontsize=10,
    )

    # Adjust layout
    fig.tight_layout(rect=[0, 0, 1, 0.95])  # Leave space for the legend
    fig.suptitle(f"Hitting Metrics Progress for Athlete ID: {athlete_id}", fontsize=16, fontweight="bold")

    # Save the figure to PDF
    pdf.savefig(fig)
    plt.close(fig)